In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.0 MB/s eta 0:00:00


In [2]:
!pip install senticnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 MB 7.8 MB/s eta 0:00:00


In [3]:
! python setup.py install

python3: can't open file '/content/setup.py': [Errno 2] No such file or directory


In [4]:
! pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.5 MB/s eta 0:00:00


In [5]:
#Importamos las librerias necesarias
from transformers import pipeline
import pandas as pd
import numpy as np
from unidecode import unidecode
from senticnet.babelsenticnet import BabelSenticNet

In [45]:
#Modelo de lxyuan
lxyuan_model = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    return_all_scores=True) #Agregue false para que el modelo solo regrese el valor más alto y probable

#Modelo de citizenlab
citizenlab_model = pipeline("text-classification",
                            model="citizenlab/twitter-xlm-roberta-base-sentiment-finetunned",
                            tokenizer="citizenlab/twitter-xlm-roberta-base-sentiment-finetunned",
                            return_all_scores=True)
# Crea una instancia de la clase BabelSenticNet para español (código de idioma 'es')
sn = BabelSenticNet('es')

def sentic_model(sentence):
    words = sentence.split()
    total_polarity = 0

    for word in words:
        try:
            polarity_value = sn.polarity_value(word)
            total_polarity += polarity_value
        except KeyError:
            # Si la palabra no está en el conjunto de datos, ignórala o puedes asignar un valor predeterminado
            pass
        except Exception as e:
            # Manejar otras excepciones si es necesario
            print(f'Ocurrió un error al procesar la palabra "{word}": {e}')

    if total_polarity >= 0:
        return 1
    elif total_polarity < 0:
        return 0

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [46]:
def procesar_resultado_citizenlab(resultado):
    # Obtener el diccionario con el score más alto
    max_score_label = max(resultado[0], key=lambda x: x['score'])

    # Verificar el label con el score más alto y devolver el resultado correspondiente
    if max_score_label['label'] == 'Positive':
        return 1
    elif max_score_label['label'] == 'Negative':
        return 0
    elif max_score_label['label'] == 'Neutral':
        # Filtrar los diccionarios con 'Positive' y 'Negative' y obtener el segundo más alto
        other_scores = [x['score'] for x in resultado[0] if x['label'] in ['Positive', 'Negative']]
        if len(other_scores) >= 2:
            second_highest_score = sorted(other_scores, reverse=True)[1]
            if max_score_label['score'] == second_highest_score:
                return 1 if 'Positive' in [x['label'] for x in resultado[0]] else 0
            else:
                return 0 if max_score_label['label'] == 'Negative' else 1
        else:
            # Si no hay suficientes scores para comparar, devolver 0
            return 0

In [48]:
def procesar_resultado_lxyuan(resultado):
    # Obtener el diccionario con el score más alto
    max_score_label = max(resultado[0], key=lambda x: x['score'])

    # Verificar el label con el score más alto y devolver el resultado correspondiente
    if max_score_label['label'] == 'positive':
        return 1
    elif max_score_label['label'] == 'negative':
        return 0
    elif max_score_label['label'] == 'neutral':
        # Filtrar los diccionarios con 'Positive' y 'Negative' y obtener el segundo más alto
        other_scores = [x['score'] for x in resultado[0] if x['label'] in ['Positive', 'Negative']]
        if len(other_scores) >= 2:
            second_highest_score = sorted(other_scores, reverse=True)[1]
            if max_score_label['score'] == second_highest_score:
                return 1 if 'Positive' in [x['label'] for x in resultado[0]] else 0
            else:
                return 0 if max_score_label['label'] == 'Negative' else 1
        else:
            # Si no hay suficientes scores para comparar, devolver 0
            return 0

In [49]:
# Cargar el archivo CSV en un DataFrame
tweets = pd.read_csv("tweets_datos.csv")

# Obtener los nombres de las columnas actuales
column_names = tweets.columns.tolist()

# Agregar una nueva fila al principio del DataFrame
tweets.loc[-1] = column_names
tweets.index = tweets.index + 1
tweets = tweets.sort_index()

# Cambiar los nombres de las columnas
tweets.columns = ["impacto", "tweet"]

In [50]:
#Agregamos columnas para cada modelo
tweets['lxyuan'] = 0

tweets['citizenlab'] = 0

tweets['sentic'] = 0

In [51]:
# Limpiar la columna "tweets"
tweets['tweet'] = tweets['tweet'].apply(lambda x: unidecode(x).lower())

# Reemplazar menciones de usuario
tweets['tweet'] = tweets['tweet'].str.replace(r'@\w+', '@user')

# Reemplazar enlaces
tweets['tweet'] = tweets['tweet'].str.replace(r'http\S+', 'link')

<ipython-input-51-b9eebdf4a59d>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets['tweet'] = tweets['tweet'].str.replace(r'@\w+', '@user')
<ipython-input-51-b9eebdf4a59d>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets['tweet'] = tweets['tweet'].str.replace(r'http\S+', 'link')


In [52]:
a = citizenlab_model("hola")
procesar_resultado(a)

1

In [ ]:
#iteracion por df para obtener clasificaciones de ambos modelos
tweets['sentic'] = tweets['tweet'].apply(lambda tw: sentic_model(tw))
tweets['lxyuan'] = tweets['tweet'].apply(lambda tw: procesar_resultado_lxyuan(lxyuan_model(tw)))
tweets['citizenlab'] = tweets['tweet'].apply(lambda tw: procesar_resultado_citizenlab(citizenlab_model(tw)))

In [54]:
tweets

,impacto,tweet,lxyuan,citizenlab,sentic
0,linf,el diputado local de sonora por el partido enc...,0,1,1
1,linf,video yo estoy de acuerdo con la filosofia de...,1,1,0
2,linf,el gobiernojalisco ya esta listo para la react...,1,1,0
3,linf,tomando te de canela en la manana a mediodia y...,0,1,1
4,linf,envivo tengo la certeza de que los militares ...,0,1,1
...,...,...,...,...,...
4995,hinf,pues ya entre al compranet y efectivamente est...,0,0,0
4996,hinf,amigos moleculas les dije desde temprano que ...,0,0,0
4997,hinf,la confirmacion que hace esta tarde the wall s...,0,0,0
4998,hinf,el dia de hoy done mi billete de la ln para la...,0,0,0


In [12]:
tweets.to_csv('tweets_clasificados.csv', index=False)